In [10]:
import requests
import pandas as pd

import sys

sys.path.append("S://Docs//Personal//MAEVE//Data//Config//")

from config import *

## Get Historical data

In [3]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=60')

df = pd.DataFrame(resp.json()['result']['XXBTZUSD'], columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
df['time'] = pd.to_datetime(df['time'], unit='s')

print(df.shape)

(720, 8)


## Get price

In [4]:
import requests

url = "https://api.kraken.com/0/public/Ticker?pair=XBTUSD"

response = requests.get(url)
data = response.json()

price = data['result']['XXBTZUSD']['c'][0]
print("Current price of BTC:", price)

Current price of BTC: 23387.00000


## Get USD balance

In [30]:
import time
import os
import requests

import urllib.parse
import hashlib
import hmac
import base64


def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()



# Read Kraken API key and secret stored in environment variables
api_url = "https://api.kraken.com"

# Attaches auth headers and returns results of a POST request


def kraken_request(uri_path, data):
    headers = {}
    headers['API-Key'] = API_KEY
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, SECRET_KEY)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req


# Construct the request and print the result
resp = kraken_request('/0/private/Balance', {
    "nonce": str(int(1000*time.time()))
})


print(resp.json())


{'error': [], 'result': {'ADA': '0.00000001', 'ICX': '0.0000000100', 'SOL': '0.0000065700', 'ETHW': '0.0000000', 'USDT': '0.00068351', 'DOT': '0.1021702000', 'SOL.S': '0.0033541100', 'XXRP': '0.00000000', 'DOT.S': '0.0000000000', 'ZUSD': '0.0042', 'XETH': '0.0000000080', 'XXBT': '0.0000099500'}}


In [31]:
resp.json()['result']['ZUSD']


'0.0042'

## Asset info

In [21]:
import requests

resp = requests.get('https://api.kraken.com/0/public/Assets?asset=XBT')

print(resp.json())


{'error': [], 'result': {'XXBT': {'aclass': 'currency', 'altname': 'XBT', 'decimals': 10, 'display_decimals': 5, 'collateral_value': 1.0, 'status': 'enabled'}}}
